In [2]:
# ================================================================================================
# 갑상선암 진단 대회 - 완전한 솔루션
# 전략: 안정적이고 재현 가능한 앙상블 기반 접근법
# ================================================================================================

import pandas as pd
import numpy as np
import warnings
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝 라이브러리
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight

# 그래디언트 부스팅
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# 불균형 데이터 처리
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

# 하이퍼파라미터 튜닝
from sklearn.model_selection import RandomizedSearchCV

warnings.filterwarnings('ignore')

# ================================================================================================
# 1. 설정 및 유틸리티 함수
# ================================================================================================

# 재현성을 위한 시드 고정
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def set_seeds(seed=42):
    """모든 라이브러리의 시드를 고정하는 함수"""
    import random
    import os
    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

class Config:
    """설정 클래스"""
    RANDOM_STATE = 42
    N_SPLITS = 5
    EARLY_STOPPING_ROUNDS = 100
    VERBOSE = False
    
    # 모델별 기본 파라미터
    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'random_state': RANDOM_STATE,
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1
    }
    
    LGB_PARAMS = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'random_state': RANDOM_STATE,
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
        'verbose': -1
    }
    
    CB_PARAMS = {
        'objective': 'Logloss',
        'random_state': RANDOM_STATE,
        'iterations': 1000,
        'learning_rate': 0.05,
        'depth': 6,
        'subsample': 0.8,
        'colsample_bylevel': 0.8,
        'reg_lambda': 0.1,
        'verbose': False
    }

set_seeds(Config.RANDOM_STATE)

# ================================================================================================
# 2. 데이터 로딩 및 기본 분석
# ================================================================================================

class DataLoader:
    """데이터 로딩 및 기본 전처리 클래스"""
    
    def __init__(self):
        self.train_data = None
        self.test_data = None
        self.feature_columns = None
        self.target_column = 'Cancer'
        
    def load_data(self, train_path='train.csv', test_path='test.csv'):
        """데이터 로딩"""
        print("📊 데이터 로딩 중...")
        
        self.train_data = pd.read_csv(train_path)
        self.test_data = pd.read_csv(test_path)
        
        # Feature columns 정의 (ID와 target 제외)
        self.feature_columns = [col for col in self.train_data.columns 
                               if col not in ['ID', self.target_column]]
        
        print(f"✅ 훈련 데이터: {self.train_data.shape}")
        print(f"✅ 테스트 데이터: {self.test_data.shape}")
        print(f"✅ Feature 개수: {len(self.feature_columns)}")
        
        return self
    
    def basic_eda(self):
        """기본 EDA 수행"""
        print("\n📈 기본 EDA 수행 중...")
        
        # 클래스 분포
        class_dist = self.train_data[self.target_column].value_counts()
        print(f"\n클래스 분포:")
        for cls, count in class_dist.items():
            pct = count / len(self.train_data) * 100
            print(f"  클래스 {cls}: {count:,}개 ({pct:.1f}%)")
        
        # 결측값 확인
        print(f"\n결측값 확인:")
        missing_train = self.train_data.isnull().sum()
        missing_test = self.test_data.isnull().sum()
        
        for col in self.feature_columns:
            train_missing = missing_train[col] if col in missing_train.index else 0
            test_missing = missing_test[col] if col in missing_test.index else 0
            if train_missing > 0 or test_missing > 0:
                print(f"  {col}: 훈련({train_missing}) 테스트({test_missing})")
        
        # 수치형/카테고리컬 변수 분리
        numeric_cols = self.train_data[self.feature_columns].select_dtypes(
            include=[np.number]).columns.tolist()
        categorical_cols = [col for col in self.feature_columns if col not in numeric_cols]
        
        print(f"\n수치형 변수: {len(numeric_cols)}개")
        print(f"카테고리컬 변수: {len(categorical_cols)}개")
        
        return numeric_cols, categorical_cols

# ================================================================================================
# 3. 전처리 파이프라인
# ================================================================================================

class FeatureProcessor:
    """특성 전처리 및 엔지니어링 클래스"""
    
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.numeric_cols = []
        self.categorical_cols = []
        
    def fit_transform(self, train_data, feature_columns, target_column='Cancer'):
        """훈련 데이터에 fit하고 transform"""
        print("\n🔧 특성 전처리 중...")
        
        X_train = train_data[feature_columns].copy()
        y_train = train_data[target_column].copy()
        
        # 결측값 사전 처리
        print("📋 결측값 처리 중...")
        
        # 수치형/카테고리컬 변수 분리
        self.numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
        self.categorical_cols = [col for col in feature_columns if col not in self.numeric_cols]
        
        # 수치형 변수 결측값 처리 (중앙값으로 대체)
        for col in self.numeric_cols:
            if X_train[col].isnull().any():
                median_val = X_train[col].median()
                X_train[col].fillna(median_val, inplace=True)
                print(f"  {col}: 결측값을 {median_val:.3f}로 대체")
        
        # 카테고리컬 변수 결측값 처리 (최빈값으로 대체)
        for col in self.categorical_cols:
            if X_train[col].isnull().any() or X_train[col].isna().any():
                mode_val = X_train[col].mode().iloc[0] if len(X_train[col].mode()) > 0 else 'Unknown'
                X_train[col].fillna(mode_val, inplace=True)
                print(f"  {col}: 결측값을 '{mode_val}'로 대체")
        
        # 1. 카테고리컬 변수 인코딩
        print("🏷️  카테고리컬 변수 인코딩 중...")
        for col in self.categorical_cols:
            le = LabelEncoder()
            # 문자열로 변환하여 안전하게 처리
            X_train[col] = X_train[col].astype(str)
            X_train[col] = le.fit_transform(X_train[col])
            self.label_encoders[col] = le
            print(f"  {col}: {len(le.classes_)}개 클래스 인코딩 완료")
        
        # 2. Feature Engineering
        print("⚙️  Feature Engineering 수행 중...")
        X_train = self._feature_engineering(X_train)
        
        # 3. 최종 수치형 변수들 확인 및 스케일링
        numeric_cols_final = X_train.select_dtypes(include=[np.number]).columns.tolist()
        
        # 무한대나 NaN 값 최종 체크
        for col in numeric_cols_final:
            # 무한대 값 처리
            inf_mask = np.isinf(X_train[col])
            if inf_mask.any():
                print(f"  {col}: {inf_mask.sum()}개 무한대 값 발견, 중앙값으로 대체")
                X_train.loc[inf_mask, col] = X_train[col][~inf_mask].median()
            
            # NaN 값 최종 처리
            nan_mask = X_train[col].isnull()
            if nan_mask.any():
                print(f"  {col}: {nan_mask.sum()}개 NaN 값 발견, 중앙값으로 대체")
                X_train.loc[nan_mask, col] = X_train[col][~nan_mask].median()
        
        # 스케일링
        print("📏 수치형 변수 스케일링 중...")
        X_train[numeric_cols_final] = self.scaler.fit_transform(X_train[numeric_cols_final])
        
        print(f"✅ 전처리 완료: {X_train.shape}")
        print(f"✅ 최종 feature 개수: {len(X_train.columns)}")
        
        return X_train, y_train
    
    def transform(self, data, feature_columns):
        """테스트 데이터 transform"""
        print("🔄 테스트 데이터 전처리 중...")
        X_test = data[feature_columns].copy()
        
        # 결측값 사전 처리 (훈련 데이터와 동일한 방식)
        # 수치형 변수 결측값 처리
        for col in self.numeric_cols:
            if X_test[col].isnull().any():
                # 훈련 데이터의 중앙값 사용 (scaler에서 추출)
                if hasattr(self.scaler, 'scale_') and col in X_test.columns:
                    # 스케일러가 fit되어 있다면 mean 값 사용
                    fill_value = 0  # 스케일링 후 평균값
                else:
                    fill_value = X_test[col].median()
                X_test[col].fillna(fill_value, inplace=True)
        
        # 카테고리컬 변수 결측값 처리
        for col in self.categorical_cols:
            if X_test[col].isnull().any() or X_test[col].isna().any():
                # 훈련 데이터에서 가장 많았던 값으로 대체 (첫 번째 클래스)
                if col in self.label_encoders:
                    most_common = self.label_encoders[col].classes_[0]
                    X_test[col].fillna(most_common, inplace=True)
                else:
                    X_test[col].fillna('Unknown', inplace=True)
        
        # 1. 카테고리컬 변수 인코딩
        for col in self.categorical_cols:
            if col in self.label_encoders:
                # 문자열로 변환
                X_test[col] = X_test[col].astype(str)
                
                # 훈련 데이터에 없던 값은 가장 빈도가 높은 값으로 처리
                unknown_mask = ~X_test[col].isin(self.label_encoders[col].classes_)
                if unknown_mask.any():
                    most_common = self.label_encoders[col].classes_[0]
                    X_test.loc[unknown_mask, col] = most_common
                    print(f"  {col}: {unknown_mask.sum()}개 미지의 값을 '{most_common}'로 대체")
                
                X_test[col] = self.label_encoders[col].transform(X_test[col])
        
        # 2. Feature Engineering
        X_test = self._feature_engineering(X_test)
        
        # 3. 최종 안전성 체크
        numeric_cols_final = X_test.select_dtypes(include=[np.number]).columns.tolist()
        
        # 무한대나 NaN 값 최종 체크
        for col in numeric_cols_final:
            # 무한대 값 처리
            inf_mask = np.isinf(X_test[col])
            if inf_mask.any():
                X_test.loc[inf_mask, col] = 0  # 스케일링된 평균값
            
            # NaN 값 최종 처리
            nan_mask = X_test[col].isnull()
            if nan_mask.any():
                X_test.loc[nan_mask, col] = 0  # 스케일링된 평균값
        
        # 4. 수치형 변수 스케일링
        X_test[numeric_cols_final] = self.scaler.transform(X_test[numeric_cols_final])
        
        print(f"✅ 테스트 데이터 전처리 완료: {X_test.shape}")
        return X_test
    
    def _feature_engineering(self, data):
        """피처 엔지니어링"""
        # 기본적인 피처 엔지니어링
        data = data.copy()
        
        # Age 그룹화 (NaN 안전 처리)
        if 'Age' in data.columns:
            # NaN이 아닌 값들만 처리
            age_mask = data['Age'].notna()
            data['Age_Group'] = 0  # 기본값
            
            if age_mask.any():
                age_groups = pd.cut(data.loc[age_mask, 'Age'], 
                                  bins=[0, 30, 50, 70, 100], 
                                  labels=[0, 1, 2, 3],
                                  include_lowest=True)
                data.loc[age_mask, 'Age_Group'] = age_groups.astype(int)
        
        # 호르몬 수치 비율 (안전한 나눗셈)
        if all(col in data.columns for col in ['TSH_Result', 'T4_Result', 'T3_Result']):
            # NaN 값 처리
            tsh_safe = data['TSH_Result'].fillna(data['TSH_Result'].median())
            t4_safe = data['T4_Result'].fillna(data['T4_Result'].median())
            t3_safe = data['T3_Result'].fillna(data['T3_Result'].median())
            
            data['TSH_T4_ratio'] = tsh_safe / (t4_safe + 1e-8)
            data['T3_T4_ratio'] = t3_safe / (t4_safe + 1e-8)
            
            # 극값 처리 (이상치 제거)
            data['TSH_T4_ratio'] = np.clip(data['TSH_T4_ratio'], 0, 10)
            data['T3_T4_ratio'] = np.clip(data['T3_T4_ratio'], 0, 5)
        
        # 결절 크기 그룹화 (NaN 안전 처리)
        if 'Nodule_Size' in data.columns:
            # NaN이 아닌 값들만 처리
            nodule_mask = data['Nodule_Size'].notna()
            data['Nodule_Size_Group'] = 0  # 기본값 (가장 작은 그룹)
            
            if nodule_mask.any():
                # 실제 데이터 범위에 맞게 bins 조정
                min_size = data['Nodule_Size'].min()
                max_size = data['Nodule_Size'].max()
                
                # 더 안전한 bins 설정
                bins = [min_size - 0.1, 1, 2, 3, max_size + 0.1]
                nodule_groups = pd.cut(data.loc[nodule_mask, 'Nodule_Size'], 
                                     bins=bins, 
                                     labels=[0, 1, 2, 3],
                                     include_lowest=True)
                data.loc[nodule_mask, 'Nodule_Size_Group'] = nodule_groups.astype(int)
        
        # 추가 안전 피처들
        if 'Age' in data.columns:
            data['Age_squared'] = data['Age'] ** 2
            data['Age_log'] = np.log1p(data['Age'])  # log(1+x) - 0 방지
        
        return data

# ================================================================================================
# 4. 모델 클래스들
# ================================================================================================

class ModelTrainer:
    """모델 훈련 및 예측 클래스"""
    
    def __init__(self, config=Config()):
        self.config = config
        self.models = {}
        self.cv_scores = {}
        
    def train_single_model(self, X_train, y_train, model_type='xgb', 
                          class_weight=None, use_smote=False):
        """단일 모델 훈련"""
        print(f"\n🤖 {model_type.upper()} 모델 훈련 중...")
        
        # 클래스 불균형 처리
        if use_smote:
            smote = SMOTE(random_state=self.config.RANDOM_STATE)
            X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
            print(f"SMOTE 적용: {X_train.shape} → {X_train_balanced.shape}")
        else:
            X_train_balanced, y_train_balanced = X_train, y_train
        
        # 모델 생성
        if model_type == 'xgb':
            params = self.config.XGB_PARAMS.copy()
            if class_weight:
                scale_pos_weight = class_weight[0] / class_weight[1]
                params['scale_pos_weight'] = scale_pos_weight
            model = xgb.XGBClassifier(**params)
            
        elif model_type == 'lgb':
            params = self.config.LGB_PARAMS.copy()
            if class_weight:
                params['class_weight'] = 'balanced'
            model = lgb.LGBMClassifier(**params)
            
        elif model_type == 'catboost':
            params = self.config.CB_PARAMS.copy()
            if class_weight:
                params['class_weights'] = class_weight
            model = cb.CatBoostClassifier(**params)
            
        elif model_type == 'rf':
            model = RandomForestClassifier(
                n_estimators=500,
                max_depth=10,
                random_state=self.config.RANDOM_STATE,
                class_weight='balanced' if class_weight else None,
                n_jobs=-1
            )
        
        # Cross Validation
        cv = StratifiedKFold(n_splits=self.config.N_SPLITS, 
                           shuffle=True, 
                           random_state=self.config.RANDOM_STATE)
        
        cv_scores = cross_val_score(model, X_train_balanced, y_train_balanced, 
                                   cv=cv, scoring='f1', n_jobs=-1)
        
        print(f"CV F1 Score: {cv_scores.mean():.5f} (+/- {cv_scores.std() * 2:.5f})")
        
        # 전체 데이터로 훈련
        model.fit(X_train_balanced, y_train_balanced)
        
        # 저장
        self.models[model_type] = model
        self.cv_scores[model_type] = cv_scores.mean()
        
        return model, cv_scores.mean()
    
    def train_all_models(self, X_train, y_train):
        """모든 모델 훈련"""
        print("\n🎯 전체 모델 훈련 시작...")
        
        # 클래스 가중치 계산
        class_weights = compute_class_weight('balanced', 
                                           classes=np.unique(y_train), 
                                           y=y_train)
        class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
        
        # 각 모델 훈련
        model_configs = [
            ('xgb', False, class_weight_dict),
            ('xgb_smote', True, None),
            ('lgb', False, class_weight_dict),
            ('lgb_smote', True, None),
            ('catboost', False, class_weight_dict),
            ('rf', False, class_weight_dict)
        ]
        
        for model_name, use_smote, class_weight in model_configs:
            try:
                base_name = model_name.replace('_smote', '')
                self.train_single_model(X_train, y_train, 
                                      model_type=base_name,
                                      class_weight=class_weight,
                                      use_smote=use_smote)
                if use_smote:
                    # SMOTE 버전을 별도로 저장
                    self.models[model_name] = self.models[base_name]
                    self.cv_scores[model_name] = self.cv_scores[base_name]
            except Exception as e:
                print(f"❌ {model_name} 훈련 실패: {e}")
        
        # 결과 요약
        print(f"\n📊 모델별 CV 성능:")
        for model_name, score in sorted(self.cv_scores.items(), 
                                       key=lambda x: x[1], reverse=True):
            print(f"  {model_name}: {score:.5f}")
    
    def create_ensemble(self, X_train, y_train, ensemble_type='voting'):
        """앙상블 모델 생성"""
        print(f"\n🔄 {ensemble_type.upper()} 앙상블 생성 중...")
        
        if len(self.models) < 2:
            print("❌ 앙상블을 위한 충분한 모델이 없습니다.")
            return None
        
        # 상위 성능 모델들 선택
        top_models = sorted(self.cv_scores.items(), 
                           key=lambda x: x[1], reverse=True)[:4]
        
        estimators = [(name, self.models[name]) for name, _ in top_models]
        
        if ensemble_type == 'voting':
            # 가중 투표 (CV 성능 기반)
            weights = [score for _, score in top_models]
            ensemble = VotingClassifier(estimators=estimators, 
                                      voting='soft', 
                                      weights=weights)
        
        # 앙상블 모델 훈련
        ensemble.fit(X_train, y_train)
        
        # CV 평가
        cv = StratifiedKFold(n_splits=self.config.N_SPLITS, 
                           shuffle=True, 
                           random_state=self.config.RANDOM_STATE)
        ensemble_scores = cross_val_score(ensemble, X_train, y_train, 
                                        cv=cv, scoring='f1', n_jobs=-1)
        
        print(f"앙상블 CV F1 Score: {ensemble_scores.mean():.5f} (+/- {ensemble_scores.std() * 2:.5f})")
        
        self.models['ensemble'] = ensemble
        self.cv_scores['ensemble'] = ensemble_scores.mean()
        
        return ensemble

# ================================================================================================
# 5. 하이퍼파라미터 튜닝
# ================================================================================================

class HyperparameterTuner:
    """하이퍼파라미터 튜닝 클래스"""
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        
    def tune_xgboost(self, X_train, y_train, n_iter=50):
        """XGBoost 하이퍼파라미터 튜닝"""
        print("\n🎯 XGBoost 하이퍼파라미터 튜닝 중...")
        
        param_dist = {
            'n_estimators': [500, 1000, 1500],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 4, 5, 6, 7],
            'subsample': [0.7, 0.8, 0.9],
            'colsample_bytree': [0.7, 0.8, 0.9],
            'reg_alpha': [0, 0.1, 0.5],
            'reg_lambda': [0.1, 0.5, 1.0]
        }
        
        xgb_model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=self.random_state,
            scale_pos_weight=7.3  # 클래스 불균형 비율
        )
        
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=self.random_state)
        
        random_search = RandomizedSearchCV(
            xgb_model, param_dist,
            n_iter=n_iter,
            scoring='f1',
            cv=cv,
            random_state=self.random_state,
            n_jobs=-1,
            verbose=1
        )
        
        random_search.fit(X_train, y_train)
        
        print(f"✅ 최적 XGB 파라미터: {random_search.best_params_}")
        print(f"✅ 최적 CV F1 Score: {random_search.best_score_:.5f}")
        
        return random_search.best_estimator_, random_search.best_params_

# ================================================================================================
# 6. 메인 실행 클래스
# ================================================================================================

class ThyroidCancerPredictor:
    """갑상선암 예측 메인 클래스"""
    
    def __init__(self):
        self.data_loader = DataLoader()
        self.processor = FeatureProcessor()
        self.trainer = ModelTrainer()
        self.tuner = HyperparameterTuner()
        
        self.X_train = None
        self.y_train = None
        self.X_test = None
        
    def run_pipeline(self, train_path='train.csv', test_path='test.csv', 
                    tune_hyperparams=False):
        """전체 파이프라인 실행"""
        print("🚀 갑상선암 예측 파이프라인 시작!")
        print("=" * 60)
        
        # 1. 데이터 로딩
        self.data_loader.load_data(train_path, test_path)
        numeric_cols, categorical_cols = self.data_loader.basic_eda()
        
        # 2. 전처리
        self.X_train, self.y_train = self.processor.fit_transform(
            self.data_loader.train_data, 
            self.data_loader.feature_columns
        )
        
        self.X_test = self.processor.transform(
            self.data_loader.test_data,
            self.data_loader.feature_columns
        )
        
        # 3. 모델 훈련
        self.trainer.train_all_models(self.X_train, self.y_train)
        
        # 4. 하이퍼파라미터 튜닝 (선택사항)
        if tune_hyperparams:
            best_xgb, best_params = self.tuner.tune_xgboost(self.X_train, self.y_train)
            self.trainer.models['xgb_tuned'] = best_xgb
            
            # 튜닝된 모델 CV 평가
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            tuned_scores = cross_val_score(best_xgb, self.X_train, self.y_train,
                                         cv=cv, scoring='f1', n_jobs=-1)
            self.trainer.cv_scores['xgb_tuned'] = tuned_scores.mean()
            print(f"튜닝된 XGB CV F1: {tuned_scores.mean():.5f}")
        
        # 5. 앙상블 생성
        ensemble_model = self.trainer.create_ensemble(self.X_train, self.y_train)
        
        return self
    
    def predict_and_submit(self, submission_path='submission.csv'):
        """예측 및 제출 파일 생성"""
        print("\n📤 예측 및 제출 파일 생성 중...")
        
        # 최고 성능 모델 선택
        best_model_name = max(self.trainer.cv_scores.items(), 
                             key=lambda x: x[1])[0]
        best_model = self.trainer.models[best_model_name]
        
        print(f"✅ 선택된 모델: {best_model_name} (CV F1: {self.trainer.cv_scores[best_model_name]:.5f})")
        
        # 예측
        predictions = best_model.predict(self.X_test)
        
        # 제출 파일 생성
        submission = pd.DataFrame({
            'ID': self.data_loader.test_data['ID'],
            'Cancer': predictions
        })
        
        submission.to_csv(submission_path, index=False)
        print(f"✅ 제출 파일 저장: {submission_path}")
        
        # 예측 분포 확인
        pred_dist = pd.Series(predictions).value_counts()
        print(f"\n예측 분포:")
        for cls, count in pred_dist.items():
            pct = count / len(predictions) * 100
            print(f"  클래스 {cls}: {count:,}개 ({pct:.1f}%)")
        
        return submission
    
    def save_models(self, save_dir='models'):
        """모델 저장"""
        save_path = Path(save_dir)
        save_path.mkdir(exist_ok=True)
        
        for model_name, model in self.trainer.models.items():
            model_path = save_path / f"{model_name}_model.pkl"
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
        
        # 전처리기도 저장
        processor_path = save_path / "processor.pkl"
        with open(processor_path, 'wb') as f:
            pickle.dump(self.processor, f)
        
        print(f"✅ 모델들이 {save_dir}에 저장되었습니다.")

# ================================================================================================
# 7. 실행 코드
# ================================================================================================

def main():
    """메인 실행 함수"""
    print("🏥 갑상선암 진단 AI 모델 개발")
    print("=" * 50)
    
    # 예측기 생성 및 실행
    predictor = ThyroidCancerPredictor()
    
    try:
        # 전체 파이프라인 실행
        predictor.run_pipeline(
            train_path='train.csv',
            test_path='test.csv',
            tune_hyperparams=False  # 시간이 많으면 True로 설정
        )
        
        # 예측 및 제출
        submission = predictor.predict_and_submit('submission.csv')
        
        # 모델 저장
        predictor.save_models('models')
        
        print("\n🎉 파이프라인 완료!")
        print("=" * 50)
        
        # 최종 결과 요약
        print("\n📊 최종 결과 요약:")
        for model_name, score in sorted(predictor.trainer.cv_scores.items(), 
                                       key=lambda x: x[1], reverse=True):
            print(f"  {model_name}: CV F1 = {score:.5f}")
        
    except FileNotFoundError as e:
        print(f"❌ 파일을 찾을 수 없습니다: {e}")
        print("💡 train.csv와 test.csv 파일이 현재 디렉터리에 있는지 확인하세요.")
        
    except ValueError as e:
        print(f"❌ 데이터 처리 오류: {e}")
        print("💡 데이터 형식이나 결측값을 확인해보세요.")
        
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생: {e}")
        print("\n🔍 상세 오류 정보:")
        import traceback
        traceback.print_exc()
        
        # 디버깅 정보 제공
        print("\n🛠️  디버깅 도움말:")
        print("1. 데이터 파일이 올바른 형식인지 확인")
        print("2. 필요한 라이브러리가 모두 설치되었는지 확인")
        print("3. 메모리가 충분한지 확인")
        print("4. Python 버전이 3.7 이상인지 확인")

# 실행
if __name__ == "__main__":
    main()

# ================================================================================================
# 8. 추가 유틸리티 함수들
# ================================================================================================

def analyze_feature_importance(model, feature_names, top_k=20):
    """특성 중요도 분석"""
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\n상위 {top_k}개 중요 특성:")
        print(importance_df.head(top_k))
        
        return importance_df
    else:
        print("이 모델은 특성 중요도를 지원하지 않습니다.")
        return None

def cross_validate_threshold(model, X_val, y_val, thresholds=None):
    """최적 임계값 찾기"""
    if thresholds is None:
        thresholds = np.arange(0.3, 0.8, 0.05)
    
    best_threshold = 0.5
    best_f1 = 0
    
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_val, y_pred)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    print(f"최적 임계값: {best_threshold:.3f} (F1: {best_f1:.5f})")
    return best_threshold, best_f1

# 사용 예시 (선택사항)
"""
# 특성 중요도 분석
if 'xgb' in predictor.trainer.models:
    analyze_feature_importance(
        predictor.trainer.models['xgb'], 
        predictor.X_train.columns
    )

# 임계값 최적화 (검증 데이터가 있는 경우)
# best_threshold, best_f1 = cross_validate_threshold(
#     predictor.trainer.models['xgb'], 
#     X_val, y_val
# )
"""

🏥 갑상선암 진단 AI 모델 개발
🚀 갑상선암 예측 파이프라인 시작!
📊 데이터 로딩 중...
✅ 훈련 데이터: (87159, 16)
✅ 테스트 데이터: (46204, 15)
✅ Feature 개수: 14

📈 기본 EDA 수행 중...

클래스 분포:
  클래스 0: 76,700개 (88.0%)
  클래스 1: 10,459개 (12.0%)

결측값 확인:

수치형 변수: 5개
카테고리컬 변수: 9개

🔧 특성 전처리 중...
📋 결측값 처리 중...
🏷️  카테고리컬 변수 인코딩 중...
  Gender: 2개 클래스 인코딩 완료
  Country: 10개 클래스 인코딩 완료
  Race: 5개 클래스 인코딩 완료
  Family_Background: 2개 클래스 인코딩 완료
  Radiation_History: 2개 클래스 인코딩 완료
  Iodine_Deficiency: 2개 클래스 인코딩 완료
  Smoke: 2개 클래스 인코딩 완료
  Weight_Risk: 2개 클래스 인코딩 완료
  Diabetes: 2개 클래스 인코딩 완료
⚙️  Feature Engineering 수행 중...
📏 수치형 변수 스케일링 중...
✅ 전처리 완료: (87159, 20)
✅ 최종 feature 개수: 20
🔄 테스트 데이터 전처리 중...
✅ 테스트 데이터 전처리 완료: (46204, 20)

🎯 전체 모델 훈련 시작...

🤖 XGB 모델 훈련 중...
CV F1 Score: 0.00946 (+/- 0.00783)

🤖 XGB 모델 훈련 중...
SMOTE 적용: (87159, 20) → (153400, 20)
CV F1 Score: 0.92438 (+/- 0.00165)

🤖 LGB 모델 훈련 중...
CV F1 Score: 0.41890 (+/- 0.01299)

🤖 LGB 모델 훈련 중...
SMOTE 적용: (87159, 20) → (153400, 20)
CV F1 Score: 0.92657 (+/- 0.00105)

🤖 CATBOOST 모델 훈련 중...

"\n# 특성 중요도 분석\nif 'xgb' in predictor.trainer.models:\n    analyze_feature_importance(\n        predictor.trainer.models['xgb'], \n        predictor.X_train.columns\n    )\n\n# 임계값 최적화 (검증 데이터가 있는 경우)\n# best_threshold, best_f1 = cross_validate_threshold(\n#     predictor.trainer.models['xgb'], \n#     X_val, y_val\n# )\n"

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import xgboost as xgb
import lightgbm as lgb

# 시드 고정
np.random.seed(42)

def simple_preprocessing(train_df, test_df):
    """초간단 전처리 - 안전하고 검증된 방법만"""
    print("🔧 단순 전처리 시작...")
    
    # Feature columns (ID와 Cancer 제외)
    feature_cols = [col for col in train_df.columns if col not in ['ID', 'Cancer']]
    
    # 훈련/테스트 데이터 분리
    X_train = train_df[feature_cols].copy()
    y_train = train_df['Cancer'].copy()
    X_test = test_df[feature_cols].copy()
    
    # 1. 카테고리컬 변수만 인코딩 (수치형은 그대로)
    categorical_cols = X_train.select_dtypes(include=['object']).columns
    label_encoders = {}
    
    for col in categorical_cols:
        le = LabelEncoder()
        
        # 훈련 데이터 인코딩
        X_train[col] = le.fit_transform(X_train[col].astype(str))
        
        # 테스트 데이터 인코딩 (새로운 값은 0으로)
        test_values = X_test[col].astype(str)
        test_encoded = []
        for val in test_values:
            if val in le.classes_:
                test_encoded.append(le.transform([val])[0])
            else:
                test_encoded.append(0)  # 새로운 값은 0
        X_test[col] = test_encoded
        
        label_encoders[col] = le
        print(f"  {col}: {len(le.classes_)}개 클래스")
    
    # 2. 결측값 단순 처리
    # 수치형: 중앙값
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        median_val = X_train[col].median()
        X_train[col].fillna(median_val, inplace=True)
        X_test[col].fillna(median_val, inplace=True)
    
    print(f"✅ 전처리 완료: Train {X_train.shape}, Test {X_test.shape}")
    return X_train, y_train, X_test

def train_simple_models(X_train, y_train):
    """간단한 모델들 훈련"""
    print("\n🤖 단순 모델 훈련...")
    
    models = {}
    cv_scores = {}
    
    # Cross Validation 설정
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 1. Random Forest (클래스 가중치만)
    print("  Random Forest 훈련 중...")
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        class_weight='balanced',
        n_jobs=-1
    )
    rf_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='f1')
    rf.fit(X_train, y_train)
    models['rf'] = rf
    cv_scores['rf'] = rf_scores.mean()
    print(f"    RF CV F1: {rf_scores.mean():.4f} ± {rf_scores.std():.4f}")
    
    # 2. XGBoost (기본 설정)
    print("  XGBoost 훈련 중...")
    # 클래스 가중치 계산
    pos_count = (y_train == 1).sum()
    neg_count = (y_train == 0).sum()
    scale_pos_weight = neg_count / pos_count
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        scale_pos_weight=scale_pos_weight,
        eval_metric='logloss'
    )
    xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='f1')
    xgb_model.fit(X_train, y_train)
    models['xgb'] = xgb_model
    cv_scores['xgb'] = xgb_scores.mean()
    print(f"    XGB CV F1: {xgb_scores.mean():.4f} ± {xgb_scores.std():.4f}")
    
    # 3. LightGBM (기본 설정)
    print("  LightGBM 훈련 중...")
    lgb_model = lgb.LGBMClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        class_weight='balanced',
        verbose=-1
    )
    lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=cv, scoring='f1')
    lgb_model.fit(X_train, y_train)
    models['lgb'] = lgb_model
    cv_scores['lgb'] = lgb_scores.mean()
    print(f"    LGB CV F1: {lgb_scores.mean():.4f} ± {lgb_scores.std():.4f}")
    
    # 최고 모델 선택
    best_model_name = max(cv_scores.items(), key=lambda x: x[1])[0]
    print(f"\n✅ 최고 모델: {best_model_name} (CV F1: {cv_scores[best_model_name]:.4f})")
    
    return models, cv_scores, best_model_name

def make_predictions(models, best_model_name, X_test, test_df):
    """예측 및 제출 파일 생성"""
    print(f"\n📤 {best_model_name} 모델로 예측 중...")
    
    best_model = models[best_model_name]
    predictions = best_model.predict(X_test)
    
    # 제출 파일 생성
    submission = pd.DataFrame({
        'ID': test_df['ID'],
        'Cancer': predictions
    })
    
    # 예측 분포 확인
    pred_counts = pd.Series(predictions).value_counts()
    print(f"\n예측 분포:")
    for cls in [0, 1]:
        count = pred_counts.get(cls, 0)
        pct = count / len(predictions) * 100
        print(f"  클래스 {cls}: {count:,}개 ({pct:.1f}%)")
    
    return submission

def main():
    """메인 실행"""
    print("🚀 단순 베이스라인 시작!")
    print("=" * 40)
    
    try:
        # 1. 데이터 로딩
        print("📊 데이터 로딩...")
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        
        print(f"  Train: {train_df.shape}")
        print(f"  Test: {test_df.shape}")
        
        # 클래스 분포 확인
        class_dist = train_df['Cancer'].value_counts()
        print(f"  클래스 0: {class_dist[0]:,}개 ({class_dist[0]/len(train_df)*100:.1f}%)")
        print(f"  클래스 1: {class_dist[1]:,}개 ({class_dist[1]/len(train_df)*100:.1f}%)")
        
        # 2. 전처리
        X_train, y_train, X_test = simple_preprocessing(train_df, test_df)
        
        # 3. 모델 훈련
        models, cv_scores, best_model_name = train_simple_models(X_train, y_train)
        
        # 4. 예측
        submission = make_predictions(models, best_model_name, X_test, test_df)
        
        # 5. 저장
        submission.to_csv('simple_submission.csv', index=False)
        print(f"✅ 제출 파일 저장: simple_submission.csv")
        
        print("\n🎯 CV 점수 요약:")
        for model_name, score in sorted(cv_scores.items(), key=lambda x: x[1], reverse=True):
            print(f"  {model_name}: {score:.4f}")
        
        print("\n💡 만약 여전히 점수가 낮다면:")
        print("1. 데이터에 문제가 있을 수 있음")
        print("2. Public/Private 분할 문제")
        print("3. 평가 지표 차이")
        print("4. 다른 참가자들도 비슷한 점수일 가능성")
        
    except Exception as e:
        print(f"❌ 오류: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

# ============================================
# 추가 디버깅용 함수들
# ============================================

def check_data_quality(train_df, test_df):
    """데이터 품질 체크"""
    print("\n🔍 데이터 품질 체크:")
    
    feature_cols = [col for col in train_df.columns if col not in ['ID', 'Cancer']]
    
    for col in feature_cols:
        train_unique = train_df[col].nunique()
        test_unique = test_df[col].nunique()
        
        # 카테고리컬 변수에서 차이가 큰 경우
        if train_df[col].dtype == 'object':
            train_set = set(train_df[col].unique())
            test_set = set(test_df[col].unique())
            only_in_test = test_set - train_set
            
            if only_in_test:
                print(f"  ⚠️  {col}: 테스트에만 있는 값 {len(only_in_test)}개")
    
    print("✅ 데이터 품질 체크 완료")

def quick_feature_importance(model, feature_names, top_k=10):
    """간단한 특성 중요도"""
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\n📊 상위 {top_k}개 중요 특성:")
        for i, row in importance_df.head(top_k).iterrows():
            print(f"  {row['feature']}: {row['importance']:.4f}")
    
# 사용 예시 (선택사항):
# check_data_quality(train_df, test_df)
# quick_feature_importance(models['xgb'], X_train.columns)

🚀 단순 베이스라인 시작!
📊 데이터 로딩...
  Train: (87159, 16)
  Test: (46204, 15)
  클래스 0: 76,700개 (88.0%)
  클래스 1: 10,459개 (12.0%)
🔧 단순 전처리 시작...
  Gender: 2개 클래스
  Country: 10개 클래스
  Race: 5개 클래스
  Family_Background: 2개 클래스
  Radiation_History: 2개 클래스
  Iodine_Deficiency: 2개 클래스
  Smoke: 2개 클래스
  Weight_Risk: 2개 클래스
  Diabetes: 2개 클래스
✅ 전처리 완료: Train (87159, 14), Test (46204, 14)

🤖 단순 모델 훈련...
  Random Forest 훈련 중...
    RF CV F1: 0.4736 ± 0.0079
  XGBoost 훈련 중...
    XGB CV F1: 0.4709 ± 0.0034
  LightGBM 훈련 중...
    LGB CV F1: 0.4765 ± 0.0037

✅ 최고 모델: lgb (CV F1: 0.4765)

📤 lgb 모델로 예측 중...

예측 분포:
  클래스 0: 40,221개 (87.1%)
  클래스 1: 5,983개 (12.9%)
✅ 제출 파일 저장: simple_submission.csv

🎯 CV 점수 요약:
  lgb: 0.4765
  rf: 0.4736
  xgb: 0.4709

💡 만약 여전히 점수가 낮다면:
1. 데이터에 문제가 있을 수 있음
2. Public/Private 분할 문제
3. 평가 지표 차이
4. 다른 참가자들도 비슷한 점수일 가능성
